<a href="https://colab.research.google.com/github/goelpravin/practice/blob/master/Cnnscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test
print("hello")

hello
